In [1]:
import torch

from roarena.corruption import CorruptionJob
from roarena.corruption import CORRUPTIONS

job = CorruptionJob('store', 'datasets', readonly=True)

In [2]:
for animal in ['mouse', 'monkey']:
    if animal=='mouse':
        model_paths = {
            'base': 'store/models/CIFAR10-G_Baseline.pt',
            'neural': 'store/models/CIFAR10-G_Mouse.pt',
        }
    if animal=='monkey':
        model_paths = {
            'base': 'store/models/VGG19_Baseline.pt',
            'neural': 'store/models/VGG19_MTL.pt',
        }
    
    accs = {}
    for label, model_path in model_paths.items():
        saved = torch.load(model_path)
        accs[label] = {0: saved['acc']}
        for severity in range(1, 6):
            accs[label][severity] = []
            for corruption in CORRUPTIONS:
                config = {
                    'model_pth': model_path,
                    'severity': severity,
                    'corruption': corruption,
                }
                key = job.configs.get_key(config)
                accs[label][severity].append(job.previews[key]['acc'])
    torch.save(accs, f'store/figs-data/{animal}_corruption.pt')